In [1]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(temperature=0, model_name="gemma2-9b-it")

# Generate the Test Set

In [22]:
from pydantic import BaseModel, Field
from typing import List

class Node(BaseModel):
    """Ground truth data for evaluation."""
    question: str = Field(description="Question to ask")
    ground_truth: str = Field(description="Ground truth data")



structured_llm = llm.with_structured_output(Node)


In [23]:
system = "You are a system to generate natural question and  ground truth by given contexts "
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | structured_llm


In [11]:
with open("../.gradio/scrappedFQA.json") as f:
    documents = json.load(f)["data"]

In [12]:
df = pd.DataFrame([f"{doc['query']}/n/n{doc['solution']}" for doc in documents], columns=["text"])

In [26]:
li = []
for data in df.sample(100)["text"].tolist():
    li.append(chain.invoke({"text": data}))

In [28]:
li[5]

Node(question='What are the advantages of Secure one/Life insurance?', ground_truth='The advantages of Secure one/Life insurance include dual policy benefit, tax advantage, and no requirement for prior medical examination or documentation.')

In [34]:
test_set = pd.DataFrame([[data.question,data.ground_truth] for data in li],columns=["question", "ground_truth"])

In [35]:
test_set

,question,ground_truth
0,Are there any charges applicable on my credit ...,"Yes, charges such as late payment charges, int..."
1,What are the steps to start investing in mutua...,"To start investing in mutual funds, you need t..."
2,What should I do if my debit card PIN was inco...,"To update the landmark, log in to Net-Banking ..."
3,What is maturity date?,The date on which the contract of insurance po...
4,What should I do if I didn't receive the cashb...,Share a screenshot or copy of the promotional ...
...,...,...
95,What details are required to change or add a n...,"Name of the nominee, Address, Date of Birth, P..."
96,What are the key considerations for taking a g...,"To pledge ornaments owned by the customer, sho..."
97,Are there any restrictions on the transactions...,"Yes, Kotak 811 Lite is a Digital Wallet accoun..."
98,What is an Investment Horizon in mutual fund?,An Investment Horizon refers to the length of ...


In [36]:
test_set.to_csv("../data/test_set.csv", index=False)

In [37]:
questions = test_set["question"].tolist()
ground_truths = test_set["ground_truth"].tolist()

# Generate the answer and content

In [39]:

from src.chatbot import vector_store_retriever,rag_chain

In [44]:
answers = []
content = []

In [41]:
vector_store =vector_store_retriever()

In [45]:
for question in questions:
    answer = rag_chain(question,chat_history=[])
    answers.append(answer)
    content.append([docs.page_content for docs in vector_store.get_relevant_documents(question)])

In [46]:
content[0]

['Are there any charges applicable on my credit card?/n/nDepending on the type of credit card you hold with us, Annual Fees or Joining Fees or both will be applicable (if any). There are no additional charges applied on for the usage of credit card. However charges are applied in following cases:\n1. Late payment charges along with applicable GST will be levied if payment is not received till the due date of your credit card bill\xa0\n2. Interest will be charged on all unpaid transactions till the date the part payment is received by us and then on the balance amount till the next statement date.\n3.Interest is also levied on all fees, charges and GST\xa0\n4. In case of part payment received towards the previous outstanding, interest will be applicable even on the fresh transactions incurred on the card account\xa0\n5. Overlimit charges of INR 500.00 plus applicable GST will be levied if you have over utilisized the credit limit offered to you on your card. Kindly click here to know th

# evaluate

In [53]:
! ollama list

NAME                       ID              SIZE      MODIFIED    
sqlcoder:latest            77ac14348387    4.1 GB    9 days ago     
nomic-embed-text:latest    0a109f422b47    274 MB    9 days ago     
llama3.2:3b                a80c4f17acd5    2.0 GB    5 weeks ago    


In [4]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [5]:
from ragas.embeddings import LangchainEmbeddingsWrapper
embedding_llm = LangchainEmbeddingsWrapper(embeddings)

In [6]:
data = {
    "question": questions,
    "ground_truth": ground_truths,
    "response": answers,
    "contexts": content
}

NameError: name 'questions' is not defined

In [61]:
with open("../data/test_set.json","w") as f:
    json.dump(data,f)

In [6]:
with open("../data/test_set.json","r") as f:
    data = json.load(f)

In [7]:
data ={
    "question": data["question"][:5],
    "ground_truth": data["ground_truth"][:5],
    "response": data["response"][:5],
    "contexts": data["contexts"][:5]
}

In [8]:
from datasets import Dataset
dataset = Dataset.from_dict(data)

In [9]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from ragas.llms import LangchainLLMWrapper
evaluator_llm = LangchainLLMWrapper(llm)

In [10]:
result = evaluate(
    dataset,
    metrics=[
        answer_relevancy,
        context_recall,
        context_precision,
        faithfulness,
    ],
    llm=evaluator_llm,
    embeddings=embedding_llm,
)

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

In [11]:
result

{'answer_relevancy': 0.5689, 'context_recall': 1.0000, 'context_precision': 0.9333, 'faithfulness': 0.9417}